1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [102]:
import pandas as pd

def get_row_number(data_frame):
    return data_frame.shape[0];

def print_columns(data_frame):
    print ("  --> Columns {}".format(data_frame.columns));

def analyze_class_balance (data_frame,feature_name,target_positive=1,target_negative=0):
    positive_number = get_row_number(data_frame[data_frame[feature_name]==target_positive]);
    negative_number = get_row_number(data_frame[data_frame[feature_name]==target_negative]);
    print ("  --> Found {} p/n: {}/{}".format(feature_name,positive_number,negative_number));
    return (positive_number,negative_number,positive_number+negative_number,feature_name);

def select_columns(data_frame,columns):
    new_data_frame = data_frame[['{}'.format(c) for c in columns]].copy(deep=True);
    return new_data_frame;


In [103]:
data_frame = pd.read_csv("train.csv");


In [104]:
data_frame.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [105]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [106]:

# Balance analysis:---------------------------------------
data_frame = pd.read_csv("train.csv")

print_columns(data_frame);
for column in data_frame.columns:
    analyze_class_balance(data_frame,column)


  --> Columns Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')
  --> Found id p/n: 0/0
  --> Found comment_text p/n: 0/0
  --> Found toxic p/n: 15294/144277
  --> Found severe_toxic p/n: 1595/157976
  --> Found obscene p/n: 8449/151122
  --> Found threat p/n: 478/159093
  --> Found insult p/n: 7877/151694
  --> Found identity_hate p/n: 1405/158166


In [107]:
# Selecting X,Y:-------------------------------------------
TARGET='threat'

X = select_columns(data_frame,['comment_text']);
Y = select_columns(data_frame,[TARGET])
(X,Y)


(                                             comment_text
 0       Explanation\nWhy the edits made under my usern...
 1       D'aww! He matches this background colour I'm s...
 2       Hey man, I'm really not trying to edit war. It...
 3       "\nMore\nI can't make any real suggestions on ...
 4       You, sir, are my hero. Any chance you remember...
 ...                                                   ...
 159566  ":::::And for the second time of asking, when ...
 159567  You should be ashamed of yourself \n\nThat is ...
 159568  Spitzer \n\nUmm, theres no actual article for ...
 159569  And it looks like it was actually you who put ...
 159570  "\nAnd ... I really don't think you understand...
 
 [159571 rows x 1 columns],
         threat
 0            0
 1            0
 2            0
 3            0
 4            0
 ...        ...
 159566       0
 159567       0
 159568       0
 159569       0
 159570       0
 
 [159571 rows x 1 columns])

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

original_comments = [c for c in X['comment_text']]

# print("  --> Original comments:\n {}".format(original_comments[:10]));

assumed_features = 1000
vectorizer = TfidfVectorizer(max_features=assumed_features)
vectorized_comments = vectorizer.fit_transform(original_comments);



In [109]:
import numpy as np

xv = pd.DataFrame([np.ravel(row.todense()) for row in vectorized_comments])
yv = Y.copy();
(xv,yv)


(        0    1    2         3    4    5    6    7    8    9    ...  990  991  \
 0       0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 1       0.0  0.0  0.0  0.392295  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 2       0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 3       0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 4       0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 ...     ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 159566  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 159567  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 159568  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 159569  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 159570  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 
         992  993  994  99

In [110]:
from sklearn.model_selection import train_test_split


# Splitting for test and train sets:-----------------------------------
# train---test----train-----test-----
x_train, x_test, y_train, y_test = train_test_split(xv,yv,train_size=0.5,shuffle=True,random_state=1)
print("X train shape={}".format(x_train.shape));
print("X test shape={}".format(x_test.shape));
print("Y train shape={}".format(y_train.shape));
print("Y test shape={}".format(y_test.shape));

# Additional class balance info:---------------------------------------
analyze_class_balance(y_test,TARGET)
analyze_class_balance(y_train,TARGET)


X train shape=(79785, 1000)
X test shape=(79786, 1000)
Y train shape=(79785, 1)
Y test shape=(79786, 1)
  --> Found threat p/n: 246/79540
  --> Found threat p/n: 232/79553


(232, 79553, 79785, 'threat')

In [111]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

clf = BalancedRandomForestClassifier(n_estimators=500,max_depth=250, random_state=0)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test);
class_report = classification_report(y_test,y_pred);
print ("BalancedRandomForestClassifier classification report: \n{}".format(class_report));

BalancedRandomForestClassifier classification report: 
              precision    recall  f1-score   support

           0       1.00      0.91      0.95     79540
           1       0.03      0.84      0.05       246

    accuracy                           0.91     79786
   macro avg       0.51      0.87      0.50     79786
weighted avg       1.00      0.91      0.95     79786



In [112]:
from sklearn.model_selection import cross_val_score


clf = BalancedRandomForestClassifier(n_estimators=500,max_depth=250, random_state=0)

cv_scores = cross_val_score(clf, x_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)
print('BalancedRandomForestClassifier cross-validation score: {}'.format(cv_score))

BalancedRandomForestClassifier cross-validation score: 0.9638911009964394


In [113]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=0.1, solver='sag')
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test);
class_report = classification_report(y_test,y_pred);
print ("Logistic regression classification report: \n{}".format(class_report));

Logistic regression classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     79540
           1       0.00      0.00      0.00       246

    accuracy                           1.00     79786
   macro avg       0.50      0.50      0.50     79786
weighted avg       0.99      1.00      1.00     79786



In [114]:
clf = LogisticRegression(C=0.1, solver='sag')

cv_scores = cross_val_score(clf, x_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)
print('Logistic regression cross-validation score: {}'.format(cv_score))

Logistic regression cross-validation score: 0.9629875270376479
